## Variables

### Import des librairies

In [1]:
import os
import urllib.parse as up
from rdflib import Graph
import code.functions as fn

### Définition des variables globales

In [2]:
# Fichiers existants
addr_ont_file = "address_ont.ttl"
ev_ont_file = "events_ont.ttl"

# Fichiers créés durant le processus
export_file = "addresses-temp.ttl"
out_file = "addresses.ttl"
local_config_file_name = "config_repo.ttl"

temp_folder = "tmp_files"
source_folder = "sources"
mapping_folder = "mappings"
wikidata_folder = "wikidata"

project_name = "voies_paris_hist" 
graphdb_url = "http://localhost:7200"

### Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier temporaire s'il n'existe pas encore pour stocker des fichiers à vocation d'être supprimés

In [5]:
temp_folder = os.path.abspath(temp_folder)
wikidata_folder = os.path.join(temp_folder, wikidata_folder)
source_folder = os.path.abspath(source_folder)
mapping_folder = os.path.abspath(mapping_folder)

fn.create_folder_if_not_exists(temp_folder)
fn.create_folder_if_not_exists(wikidata_folder)

local_config_file_name = os.path.join(temp_folder, local_config_file_name)

## Création d'un répertoire local

Création du répertoire local dans GraphDB. Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne.

In [ ]:
fn.create_config_local_repository_file(local_config_file_name, project_name)
url = f"{graphdb_url}/rest/repositories"
curl_cmd_local = fn.get_curl_command("POST", url, content_type="multipart/form-data", form=f"config=@{local_config_file_name}")
os.system(curl_cmd_local)

## Traitements pour créer / importer des données

### Requêtes SPARQL pour construire le graphe des rues de Paris à partir de Wikidata

:warning: Impossible de chaîner les requêtes sur le endpoint de Wikidata, d'où la séparation de celles-ci sous plusieurs variables

Note : `wd:Q16024163` décrit l'élément `réseau viaire de Paris` donc le triplet `?street p:P361 [ps:P361 wd:Q16024163].` permet de sélectionner les élements qui font partie de ce réseau. `wd:Q107311481` est similaire à la différence qu'il décrit l'élément `anciennes voies de Paris`.

* `query_1` : sélection des voies de Paris avec leur label, leurs potentiels alias, ainsi que leur localisation (arrondissements et/ou quartiers)
* `query_2` : sélection des voies de Paris avec l'ensemble de leur nom officiel avec si elles existent les dates de début et de fin de validité du nom officiel pour la voie
* `query_3` : sélection (si elles existent) des dates de création et de disparition des voies de Paris
* `query_4` : sélection des arrondissements de Paris avec leur label, leurs potentiels alias et leur localisation (qui est normalement Paris, ie `wd:Q90`)
* `query_5` : similaire à `query_4` mais repose sur les quartiers de Paris (leur localisation est normalement un arrondissement)
* `query_6` : récupération de données sur la ville de Paris (label, alias)


In [ ]:
# Get streets of Paris, with label, altLabel and their locations
query1 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

CONSTRUCT {
 ?street a addr:Landmark;
           addr:isLandmarkType addr:Thoroughfare;
           rdfs:label ?streetLabel;
           skos:altLabel ?streetAltLabel;
           addr:within ?loc ;
           addr:hasAttribute ?officialNameAttr.
}
WHERE {
  { ?street p:P361 [ps:P361 wd:Q16024163]. }
  UNION
  { ?street p:P361 [ps:P361 wd:Q107311481]. }
  ?street wdt:P131 wd:Q169293.
  ?street wdt:P131 ?loc.
  ?street rdfs:label ?streetLabel.
  FILTER (LANG(?streetLabel) = "fr")
  OPTIONAL {?street skos:altLabel ?streetAltLabel. FILTER (LANG(?streetAltLabel) = "fr")}
}
"""

# Get official name of streets and its history
query2 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

CONSTRUCT {
  ?street addr:hasAttributeVersion ?officialNameSt.
  ?officialNameSt a addr:AttributeVersion; addr:isAttributeType addr:NameAttribute; addr:value ?officialName.
  ?startEvent a addr:Event; addr:eventTimeValue ?startDateValue; addr:eventTimePrecision ?startDatePrec.
  ?endEvent a addr:Event; addr:eventTimeValue ?endDateValue; addr:eventTimePrecision ?endDatePrec.
  ?startChange a addr:AttributeChange; addr:isChangeType addr:NameChange; addr:dependsOn ?startEvent; addr:after ?officialNameSt.
  ?endChange a addr:AttributeChange; addr:isChangeType addr:NameChange; addr:dependsOn ?endEvent; addr:before ?officialNameSt.
}
WHERE {
  { ?street p:P361 [ps:P361 wd:Q16024163]. }
  UNION
  { ?street p:P361 [ps:P361 wd:Q107311481]. }
  ?street wdt:P131 wd:Q169293.
    ?street p:P1448 ?officialNameSt. 
    ?officialNameSt ps:P1448 ?officialName.
    OPTIONAL{?officialNameSt pqv:P580 [wikibase:timeValue ?startDateValue; wikibase:timePrecision ?startDatePrec]}
    OPTIONAL{?officialNameSt pqv:P582 [wikibase:timeValue ?endDateValue; wikibase:timePrecision ?endDatePrec]}
    #FILTER (LANG(?officialName) = "fr")
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())) AS ?startEvent)
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())) AS ?startChange)
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())) AS ?endEvent)
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())) AS ?endChange)
}
"""

# Get creation and dissolution dates of streets
query3 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

CONSTRUCT {
  ?startEvent a addr:Event; addr:eventTimeValue ?creationTimeValue; addr:eventTimePrecision ?creationTimePrecision.
  ?endEvent a addr:Event; addr:eventTimeValue ?dissolutionTimeValue; addr:eventTimePrecision ?dissolutionTimePrecision.
  ?startChange a addr:LandmarkChange; addr:isChangeType addr:Creation; addr:appliedTo ?street; addr:dependsOn ?startEvent.
  ?endChange a addr:LandmarkChange; addr:isChangeType addr:Dissolution; addr:appliedTo ?street; addr:dependsOn ?endEvent.
}
WHERE {
  { ?street p:P361 [ps:P361 wd:Q16024163]. }
  UNION
  { ?street p:P361 [ps:P361 wd:Q107311481]. }
  ?street wdt:P131 wd:Q169293.
    OPTIONAL { ?street p:P571 [psv:P571 [wikibase:timeValue ?creationTimeValue; wikibase:timePrecision ?creationTimePrecision]]. }
    OPTIONAL { ?street p:P576 [psv:P576 [wikibase:timeValue ?dissolutionTimeValue; wikibase:timePrecision ?dissolutionTimePrecision]]. }

    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())) AS ?startEvent)
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())) AS ?startChange)
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#EV_", STRUUID())) AS ?endEvent)
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#CH_", STRUUID())) AS ?endChange)
}
"""

# Get related data of municipal arrondissements of Paris
query4 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

CONSTRUCT {
  ?arrdt a addr:Landmark;
           addr:isLandmarkType addr:District;
           rdfs:label ?arrdtLabel;
           skos:altLabel ?arrdtAltLabel;
           addr:within wd:Q90;
           addr:startDate ?startDate.
}
WHERE {
  ?arrdt wdt:P31 wd:Q702842; p:P131 [ps:P131 wd:Q90]; rdfs:label ?arrdtLabel; skos:altLabel ?arrdtAltLabel.
  FILTER(LANG(?arrdtLabel) = "fr" && LANG(?arrdtAltLabel) = "fr")
  OPTIONAL {?arrdt wdt:P571 ?startDate}
}
"""

# Get related data of quartiers of Paris
query5 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

CONSTRUCT {
  ?quartier a addr:Landmark;
           addr:isLandmarkType addr:District;
           rdfs:label ?quartierLabel;
           skos:altLabel ?quartierAltLabel;
           addr:within ?loc.
}
WHERE {
  ?quartier wdt:P31 wd:Q252916; rdfs:label ?quartierLabel; skos:altLabel ?quartierAltLabel; p:P131 [ps:P131 ?loc].
  FILTER(LANG(?quartierLabel) = "fr" && LANG(?quartierAltLabel) = "fr")
}
"""

# Get related data of Paris
query6 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

CONSTRUCT {
  ?paris a addr:Landmark;
           addr:isLandmarkType addr:City;
           rdfs:label ?parisLabel;
           skos:altLabel ?parisAltLabel.
}
WHERE {
  BIND (wd:Q90 AS ?paris)
  ?paris rdfs:label ?parisLabel; skos:altLabel ?parisAltLabel.
  FILTER(LANG(?parisLabel) = "fr" && LANG(?parisAltLabel) = "fr")
}
"""

wiki_queries = [query1, query2, query3, query4, query5, query6]

#### Traitements après l'import de données

* `loc_query_1` : 
    * ajoute un élément de classe `addr:Attribute` relatif au nom de la voie qui permet d'avoir l'ensemble des noms officiels que cette dernière a porté durant son existence. Il est lié à la voie via le prédicat `addr:hasAttribute` ;
    * `BIND(URI(...) AS ?z)` est appelé deux fois afin que `?attrName` soit unique pour chaque rue (sinon il possède la même valeur tout le temps).
* `loc_query_2` : lors de la construction du graphe via les requêtes de la section précédente, les versions des noms sont liées directement aux voies (`?voie ?aPourVersionDeNom ?nomVersion`). Cette requête supprime ce triplet et associe `nomVersion` a l'élément crée grâce à `loc_query_1`. Ainsi, on a : `?voie ?aPourAttributDeNom ?nomAttribut. ?nomAttribut ?aPourVersionDeNom ?nomVersion.`
* `loc_query_3` : fusion d'éléments de type `addr:Event` qui sont équivalents.

In [ ]:
# Get streets of Paris with the history of their official name

loc_query_1 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

INSERT {
    ?street addr:hasAttribute ?attrName.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
}
WHERE {
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#AN_", STRUUID())) AS ?attrName)
    ?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare.
    BIND(URI(CONCAT("http://rdf.geohistoricaldata.org/address#AN_", STRUUID())) AS ?x)
}
"""

loc_query_2 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

DELETE {
     ?street addr:hasAttributeVersion ?attrNameVersion.
     ?attrNameVersion addr:isAttributeType addr:NameAttribute.
}
INSERT {
    ?attrName addr:version ?attrNameVersion.
    ?change addr:appliedTo ?attrName.
}
WHERE {
    ?attrNameVersion a addr:AttributeVersion; addr:isAttributeType addr:NameAttribute.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
    ?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttributeVersion ?attrNameVersion; addr:hasAttribute ?attrName.
    ?change a addr:AttributeChange; (addr:before|addr:after) ?attrNameVersion.
}
"""

loc_query_3 = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX ofn: <http://www.ontotext.com/sparql/functions/>

DELETE {
    ?change2 ?p ?o.
    ?event2 ?pe ?oe.
    ?se ?pe ?event2.
}
INSERT {
    ?change1 addr:after ?attrNameVersion2.
}
WHERE {
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion1, ?attrNameVersion2.
    ?attrNameVersion1 a addr:AttributeVersion.
    ?attrNameVersion2 a addr:AttributeVersion.
    FILTER (?attrNameVersion1 != ?attrNameVersion2)
    ?change1 a addr:AttributeChange; addr:before ?attrNameVersion1; addr:dependsOn [a addr:Event; addr:eventTimeValue ?event1TimeVal; addr:eventTimePrecision ?event1TimePrec].
    ?change2 a addr:AttributeChange; addr:after ?attrNameVersion2; addr:dependsOn ?event2.
    ?event2 a addr:Event; addr:eventTimeValue ?event2TimeVal; addr:eventTimePrecision ?event2TimePrec.
    BIND(ofn:asDays(?event2TimeVal - ?event1TimeVal) AS ?diffTime)
    FILTER(?diffTime >= 0.0)
    FILTER ((?diffTime <= 1.0 && ?event1TimePrec = 11 && ?event2TimePrec = 11) ||
         (?diffTime <= 32.0 && ?event1TimePrec = 10 && ?event2TimePrec = 10) ||
          (?diffTime <= 366.0 && ?event1TimePrec = 9 && ?event2TimePrec = 9) ||
          (?diffTime <= 366.0*10 && ?event1TimePrec = 8 && ?event2TimePrec = 8) ||
          (?diffTime <= 366.0*100 && ?event1TimePrec = 7 && ?event2TimePrec = 7) 
         )
        ?change2 ?p ?o.
        {?event2 ?pe ?oe}UNION{?se ?pe ?event2}
}
"""

loc_queries = [loc_query_1, loc_query_2, loc_query_3] 

### Extraction des données des voies de Paris à partir Wikidata

* Création d'un fichier `ttl` dans le dossier temporaire qui stocke les résultats des requêtes de la section Wikidata
* Lancement des requêtes sur le endpoint de Wikidata et récupération des résultats

In [ ]:
abs_export_file = os.path.abspath(os.path.join(temp_folder, export_file))

In [ ]:
g = Graph()
for query in wiki_queries:
    keep_on_query = True
    max_loop_nb = 100
    loop_nb = 0
    limit = 500
    offset = 0
    while keep_on_query and loop_nb <= max_loop_nb:
        q = query + f'\nORDER BY ?street LIMIT {limit} OFFSET {offset}'
        query_result = fn.get_construct_query_wikidata(q)
        offset += limit
        loop_nb += 1
        if len(query_result) == 0:
            keep_on_query = False
        else:   
            g += query_result

g.serialize(destination=abs_export_file)

### Import des données sur le répertoire

#### Vidage du répertoire dont l'id est `project_name` pour y stocker les données récupérées


In [ ]:
url = f"{graphdb_url}/repositories/{project_name}/statements"
cmd = fn.get_curl_command("DELETE", url, content_type="application/x-turtle")
os.system(cmd)

#### Import des ontologies

In [ ]:
abs_addr_ont_file = os.path.abspath(addr_ont_file)
abs_ev_ont_file = os.path.abspath(ev_ont_file)
                              
url = f"{graphdb_url}/repositories/{project_name}/statements"
cmd_1 = fn.get_curl_command("POST", url, content_type="application/x-turtle", local_file=abs_addr_ont_file)
cmd_2 = fn.get_curl_command("POST", url, content_type="application/x-turtle", local_file=abs_ev_ont_file)

os.system(cmd_1)
os.system(cmd_2)

#### Import des données récupérées via Wikidata

In [ ]:
url = f"{graphdb_url}/repositories/{project_name}/rdf-graphs/wikidata"
cmd = fn.get_curl_command("POST", url, content_type="application/x-turtle", local_file=abs_export_file)
os.system(cmd)

#### Nettoyage des données

In [ ]:
url = f"{graphdb_url}/repositories/{project_name}/statements"
for loc_query in loc_queries:
    query_encoded = up.quote(loc_query)
    cmd = fn.get_curl_command("POST", url, content_type="application/x-www-form-urlencoded", post_data=f"update={query_encoded}")
    os.system(cmd)

### Requêtes pour faire des sélections dans le graphe

In [ ]:
# Sélection des noms qu'a pris la place de la Nation durant sa vie
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

select ?street ?name where { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion addr:value ?name.
}	
""" 

# Sélection de l'historique des noms de la place de la Nation avec les dates
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

select ?street ?change ?event ?timeValue ?timePrecision ?beforeVersion ?afterVersion where { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute.
    ?change addr:appliedTo ?attrName; addr:dependsOn ?event.
    ?event a addr:Event.
    OPTIONAL {?event addr:eventTimeValue ?timeValue}
    OPTIONAL {?event addr:eventTimePrecision ?timePrecision}
    OPTIONAL {?change addr:before [addr:value ?beforeVersion]}
    OPTIONAL {?change addr:after [addr:value ?afterVersion]}
}	
"""

# Sélection du nom de la place de la Nation à une date donnée
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

select ?street ?name ?selectTime where { 
	?street a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; addr:hasAttribute ?attrName; (rdfs:label|skos:altLabel) "place de la Nation"@fr.
    BIND("1782-01-08T00:00:00"^^xsd:dateTime AS ?selectTime)
    ?attrName a addr:Attribute; addr:isAttributeType addr:NameAttribute; addr:version ?attrNameVersion.
    ?attrNameVersion a addr:AttributeVersion; addr:value ?name.
    ?change1 addr:before ?attrNameVersion; addr:dependsOn ?event1.
    ?change2 addr:after ?attrNameVersion; addr:dependsOn ?event2.
    ?event1 a addr:Event.
    ?event2 a addr:Event.
    OPTIONAL {?event1 addr:eventTimeValue ?timeValue1}
    OPTIONAL {?event1 addr:eventTimePrecision ?timePrecision1}
    OPTIONAL {?event2 addr:eventTimeValue ?timeValue2}
    OPTIONAL {?event2 addr:eventTimePrecision ?timePrecision2}
    FILTER ((?timeValue1 >= ?selectTime && ?timeValue2 <= ?selectTime) ||
        (?timeValue1 >= ?selectTime && !BOUND(?timeValue2)) ||
        (?timeValue2 <= ?selectTime && !BOUND(?timeValue1))
    ) 
}	
"""

Pour chaque élément Wikidata décrivant une rue de Paris, on extrait un fichier ttl le décrivant.
Ce processus peut prendre du temps (environ 1 heure).

In [7]:
qid_variable = "qid"

query = f"""
SELECT DISTINCT ?{qid_variable}
WHERE {{
{{?{qid_variable} p:P361 [ps:P361 wd:Q16024163].}}
UNION
{{?{qid_variable} p:P361 [ps:P361 wd:Q107311481].}}
}}
"""

fn.get_ttl_files_from_wikidata_query(query, qid_variable, wikidata_folder, flavor="full")